# CS 4372.501.F22-Assignment 4


## Noah Khan and Ian Lunt

In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pandas as pd
import random

import time
import matplotlib.pylab as plt
import tensorflow_hub as hub

import datetime
import scipy.sparse

In [3]:
import pathlib
dataset_url = "https://raw.githubusercontent.com/noahk587/CS-4372-Assignment-4-Sentiment-Analysis/main/amazon_cells_labelled.txt"

In [4]:
dataset = pd.read_csv(dataset_url,delimiter = "\t",header = None)

In [5]:

dataset.columns = ["Review","Sentiment"]

In [6]:
dataset.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


## Gaussian Naive Bayes

In [7]:
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB

In [8]:

wordvec = CountVectorizer().fit_transform(dataset["Review"])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(wordvec, dataset["Sentiment"], test_size=0.2, random_state=42)

In [10]:
gnb = GaussianNB()
model = gnb.fit(X_train.toarray(), y_train)


In [11]:
from sklearn.metrics import classification_report, accuracy_score

predictions = model.predict(X_test.toarray())
predicted_probas = model.predict_proba(X_test.toarray())



print(classification_report(y_test, predictions)) # predictions contain predicted values (derived from probability with 0.5 threshold)
print('Predicted labels: ', predictions)
print('Accuracy: ', accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       0.75      0.59      0.66        93
           1       0.70      0.83      0.76       107

    accuracy                           0.72       200
   macro avg       0.73      0.71      0.71       200
weighted avg       0.73      0.72      0.72       200

Predicted labels:  [0 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 0
 1 0 0 0 0 1 0 1 1 1 0 0 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0
 1 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1
 1 0 1 1 1 0 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0
 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1]
Accuracy:  0.72


## Tensor Flow LTSM

In [136]:
X_train = dataset.iloc[y_train.index]["Review"]
X_test = dataset.iloc[y_test.index]["Review"]


In [93]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(X_train)

In [211]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(encoder)

model.add(    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        #output_dim=64,
        output_dim = 128,
        # Use masking to handle the variable sequence lengths
        mask_zero=True))

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))
model.add(tf.keras.layers.Dense(128, activation='relu'))

#model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
#model.add(tf.keras.layers.Dense(64, activation='relu'))


tf.keras.layers.Dropout(0.1)
model.add(tf.keras.layers.Dense(1))




In [212]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [213]:
history = model.fit(X_train,y_train, epochs=15)
#accuarcy with no dropout 0.9800
#accuracy with dropout 0.1 after dense relu = 0.9762
#accuracy with 128 output dim/ LSTM and dense layers = .99

## 15 epochs
#accuarcy with no dropout 0.9463
#accuracy with dropout 0.1 after dense relu = 0.9463
#accuracy with 128 output dim/ LSTM and dense layers = 0.9850

## 10 epochs

#accuracy with 128 output dim/ LSTM and dense layers = 0.9275

Epoch 1/15
25/25 [==============================] - 13s 183ms/step - loss: 0.6927 - accuracy: 0.5088 - val_loss: 0.6921 - val_accuracy: 0.4650
Epoch 2/15
25/25 [==============================] - 0s 16ms/step - loss: 0.6901 - accuracy: 0.5088 - val_loss: 0.6910 - val_accuracy: 0.4650
Epoch 3/15
25/25 [==============================] - 2s 76ms/step - loss: 0.6862 - accuracy: 0.5088 - val_loss: 0.6878 - val_accuracy: 0.4650
Epoch 4/15
25/25 [==============================] - 0s 17ms/step - loss: 0.6781 - accuracy: 0.5088 - val_loss: 0.6804 - val_accuracy: 0.4650
Epoch 5/15
25/25 [==============================] - 7s 272ms/step - loss: 0.6579 - accuracy: 0.5088 - val_loss: 0.6622 - val_accuracy: 0.4650
Epoch 6/15
25/25 [==============================] - 0s 16ms/step - loss: 0.6067 - accuracy: 0.5638 - val_loss: 0.6344 - val_accuracy: 0.5800
Epoch 7/15
25/25 [==============================] - 1s 49ms/step - loss: 0.5065 - accuracy: 0.7600 - val_loss: 0.6160 - val_accuracy: 0.7200
Epoch 8/15

In [214]:
test_loss, test_acc = model.evaluate(X_test,y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

# accuarcy with no dropout 0.8299999833106995
# accuracy with dropout 0.1 after dense relu = 0.8100000023841858
#accuracy with 128 output dim/ LSTM and dense layers = 0.8399999737739563

## 15 epochs
#accuarcy with no dropout  0.8150
#accuracy with dropout 0.1 after dense relu = 0.824999988079071
#accuracy with 128 output dim/ LSTM and dense layers =  0.8349999785423279

## 10 epochs

#accuracy with 128 output dim/ LSTM and dense layers =  0.8050

7/7 [==============================] - 1s 159ms/step - loss: 1.0134 - accuracy: 0.8250
Test Loss: 1.0134401321411133
Test Accuracy: 0.824999988079071


In [122]:
y_pred = model.predict(X_test)

7/7 [==============================] - 0s 5ms/step


## Model Incorrect Predictions

In [156]:
X_test_reset = X_test.reset_index()["Review"]
y_test_reset = y_test.reset_index()["Sentiment"]
incorrect = 0
for i in range(0,200):
    if incorrect > 10:
      break
    if (np.sign(y_pred[i]) == 1 and y_test_reset[i] == 0) or (np.sign(y_pred[i]) == -1 and y_test_reset[i] == 1) :
      print(str(y_pred[i]) + " " + X_test_reset[i])
      incorrect += 1

[0.2047101] Lousy product.
[-21.733627] The pairing of the two devices was so easy it barely took a couple minutes before I started making calls with the voice dialing feature.
[-11.091321] They have been around for years and carries the highest quality of anti-glare screen protector that I have found to date.
[-0.40059972] The noise shield is incrediable.
[-0.8794715] One of my favorite purchases ever.
[0.52228457] Reception is terrible and full of static.
[-1.2731571] Also its slim enough to fit into my alarm clock docking station without removing the case.
[13.301127] This phone tries very hard to do everything but fails at it's very ability to be a phone.
[0.64403516] My experience was terrible..... This was my fourth bluetooth headset, and while it was much more comfortable than my last Jabra (which I HATED!!!
[-1.6294961] It does everything the description said it would.
[-7.9603553] I have yet to run this new battery below two bars and that's three days without charging.


In [209]:
incorrect = 0
for i in range(0,200):
    if incorrect > 10:
      break
    if (predictions[i] != y_test_reset[i]):
      print(str(predictions[i]) + " " + X_test_reset[i])
      incorrect += 1

0 Thanks again to Amazon for having the things I need for a good price!
0 The tracfonewebsite is user friendly and makes it easy toactivate, etc.
0 Very good stuff for the price.
1 Do Not Buy for D807...wrongly advertised for D807.
1 Lousy product.
0 The pairing of the two devices was so easy it barely took a couple minutes before I started making calls with the voice dialing feature.
0 They have been around for years and carries the highest quality of anti-glare screen protector that I have found to date.
1 I did not bother contacting the company for few dollar product but I learned the lesson that I should not have bought this form online anyway.
1 You also cannot take pictures with it in the case because the lense is covered.
0 Otherwise, easy to install and use, clear sound.
1 Reception is terrible and full of static.
